<a href="https://colab.research.google.com/github/wakhunguadams/Data_science/blob/main/amazonfoodreviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing kaggle for direct loading of datasets to colabs
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#mount the google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!mkdir ~/.kaggle

In [4]:
! cp /content/drive/MyDrive/API_TOKENS/kaggle.json ~/.kaggle/

In [5]:
#change permissions to allow only users to read
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
#downloading the dataset
! kaggle datasets download snap/amazon-fine-food-reviews

100% 242M/242M [00:13<00:00, 20.2MB/s]
100% 242M/242M [00:13<00:00, 18.6MB/s]


In [7]:
! mkdir datasets

In [8]:
! ls

amazon-fine-food-reviews.zip  datasets	drive  sample_data


In [9]:
!cp /content/amazon-fine-food-reviews.zip /content/datasets/amazon-fine-food-reviews.zip

In [10]:
#unziping the dataset
!unzip -q /content/datasets/amazon-fine-food-reviews.zip -d /content/datasets

In [11]:
!rm /content/datasets/amazon-fine-food-reviews.zip

In [12]:
import nltk
import string
import re  # Import re for regular expressions

import pandas as pd
import numpy as np
import plotly.express as px  # Import for interactive visualizations
import plotly.graph_objects as go  # Import for interactive visualizations
import seaborn as sns  # Import for enhanced plotting
import matplotlib.pyplot as plt  # Import for plotting
import tensorflow as tf

from wordcloud import WordCloud  # Import WordCloud from wordcloud package
from nltk.corpus import stopwords  # Import stopwords from NLTK corpus
from nltk.stem import WordNetLemmatizer  # Import WordNetLemmatizer from NLTK stem module
from nltk.tokenize import word_tokenize  # Import word_tokenize from NLTK tokenize module
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer  # Import for text vectorization
from sklearn.linear_model import LogisticRegression  # Import for logistic regression model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # Import for model evaluation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [13]:
# read the data using pandas.read_csv()
df=pd.read_csv("/content/datasets/Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


EDA

In [14]:
df.shape

(568454, 10)

In [15]:
df=df.sample(frac=0.05,random_state=42)

In [16]:
df.shape

(28423, 10)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28423 entries, 165256 to 209331
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Id                      28423 non-null  int64 
 1   ProductId               28423 non-null  object
 2   UserId                  28423 non-null  object
 3   ProfileName             28423 non-null  object
 4   HelpfulnessNumerator    28423 non-null  int64 
 5   HelpfulnessDenominator  28423 non-null  int64 
 6   Score                   28423 non-null  int64 
 7   Time                    28423 non-null  int64 
 8   Summary                 28422 non-null  object
 9   Text                    28423 non-null  object
dtypes: int64(5), object(5)
memory usage: 2.4+ MB


In [18]:
df.isnull().sum()

Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   1
Text                      0
dtype: int64

In [19]:
df.dropna(inplace=True)

In [20]:
df.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,28422.000000,28422.000000,28422.000000,28422.000000,2.842200e+04
mean,284793.069664,1.765393,2.244916,4.182394,1.295726e+09
std,164826.330209,7.345408,7.969423,1.309317,4.833541e+07
min,3.000000,0.000000,0.000000,1.000000,9.482400e+08
25%,141784.500000,0.000000,0.000000,4.000000,1.270080e+09
50%,283462.000000,0.000000,1.000000,5.000000,1.310688e+09
75%,427618.750000,2.000000,2.000000,5.000000,1.332547e+09
max,568425.000000,454.000000,462.000000,5.000000,1.351210e+09


In [21]:
df.describe(include="object")

,ProductId,UserId,ProfileName,Summary,Text
count,28422,28422,28422,28422,28422
unique,14211,24580,23125,23475,27149
top,B007JFMH8M,A1YUL9PCJR3JTY,"O. Brown ""Ms. O. Khannah-Brown""",Delicious,"This review will make me sound really stupid, ..."
freq,46,25,25,126,13


In [22]:
df["HelpfulnessNumerator"].nunique()

96

In [23]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=df['HelpfulnessNumerator'], name='Numerator'))
fig.add_trace(go.Histogram(x=df['HelpfulnessDenominator'], name='Denominator'))

fig.update_layout(
    barmode='overlay',
    xaxis_title='Helpfulness',
    yaxis_title='Count',
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white'
)

fig.show()

In [24]:
# Create the histogram trace
trace = go.Histogram(
    x=df["Score"],
    marker=dict(
        color='#8A2BE2',
        line=dict(
            color='white',  # Set the outline color of the bars to white
            width=1.5  # Set the outline width of the bars
        )
    ),
    # Customize the bar size
    nbinsx=5,  # Set the number of bins to 5
)

# Create the layout with gray background
layout = go.Layout(
    title='Distribution of Scores',
    xaxis=dict(title='Score'),
    yaxis=dict(title='Frequency'),
    plot_bgcolor='gray',
    paper_bgcolor='gray',
    font_color='white'
)
fig = go.Figure(data=[trace], layout=layout)
fig.show()

In [25]:
fig = px.line(df, x='Time', y=df.index, color_discrete_sequence=['#8A2BE2'])

fig.update_layout(
    plot_bgcolor='gray',
    paper_bgcolor='gray',
    font_color='white'
)

fig.show()

In [26]:
summary_text = ' '.join(df['Summary'].astype(str))
# Create a word cloud with black background color
wordcloud = WordCloud(width=800, height=400, max_words=50, background_color='black').generate(summary_text)

# Create a Figure object and add a Wordcloud trace
fig = go.Figure(go.Image(z=wordcloud.to_array()))
fig.update_layout(title='Most Frequent Summary Words or Phrases',
                  xaxis={'visible': False},
                  yaxis={'visible': False},
                  plot_bgcolor='black')

# Display the word cloud
fig.show()

In [27]:
# Calculate word frequencies
word_counts = summary_text.split()
word_counts = pd.Series(word_counts).value_counts().head(10)

# Create the bar plot using Plotly
fig = px.bar(word_counts, x=word_counts.index, y=word_counts.values,
             labels={'x': 'Words', 'y': 'Frequency'},
             title='Top 10 Most Frequent Summary Words or Phrases',
             template='plotly')

# Customize the bar colors
fig.update_traces(marker_color='#8A2BE2')

# Rotate x-axis labels
fig.update_layout(xaxis_tickangle=-45)

# Show the plot
fig.show()

Data Preprocessing

In [28]:
def clean_text(data, column_name):
    # Convert data to string format
    data[column_name] = data[column_name].astype(str)
    # Lowercasing
    data[column_name] = data[column_name].str.lower()

    # Removal of special characters and punctuation
    data[column_name] = data[column_name].apply(lambda text: re.sub(r'[^\w\s]', '', text))

    # Removal of numbers
    data[column_name] = data[column_name].apply(lambda text: re.sub(r'\d+', '', text))

    # Tokenization
    data[column_name] = data[column_name].apply(lambda text: word_tokenize(text))

    # Stopword removal
    stop_words = set(stopwords.words('english'))
    data[column_name] = data[column_name].apply(lambda tokens: [word for word in tokens if word not in stop_words])

    # Lemmatization
    nltk.download('wordnet')  # Download WordNet resource
    lemmatizer = WordNetLemmatizer()
    data[column_name] = data[column_name].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])
    # Join the tokens back to text
    data[column_name] = data[column_name].apply(lambda tokens: ' '.join(tokens))

    return data

In [29]:
cleaned_df = clean_text(df,"Text")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [30]:
try:
    cleaned_df.to_csv('cleaned_df.csv', index=False)
    print("Cleaned Data Saved Successfully as cleaned_data.csv !!")
except Exception as e:
    print("Error occurred while saving cleaned data:", str(e))

Cleaned Data Saved Successfully as cleaned_data.csv !!


In [31]:
cleaned_df=pd.read_csv("/content/cleaned_df.csv")
cleaned_df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,165257,B000EVG8J2,A1L01D2BD3RKVO,"B. Miller ""pet person""",0,0,5,1268179200,Crunchy & Good Gluten-Free Sandwich Cookies!,tried couple brand glutenfree sandwich cooky b...
1,231466,B0000BXJIS,A3U62RE5XZDP0G,Marty,0,0,5,1298937600,great kitty treats,cat love treat ever cant find house pop top bo...
2,427828,B008FHUFAU,AOXC0JQQZGGB6,Kenneth Shevlin,0,2,3,1224028800,COFFEE TASTE,little le expected tends muddy taste expected ...
3,433955,B006BXV14E,A3PWPNZVMNX3PA,rareoopdvds,0,1,2,1335312000,So the Mini-Wheats were too big?,first frosted miniwheats original size frosted...
4,70261,B007I7Z3Z0,A1XNZ7PCE45KK7,Og8ys1,0,2,5,1334707200,Great Taste . . .,want congratulate graphic artist putting entir...


In [32]:
cleaned_df["Score"].value_counts()

5    18134
4     4034
1     2596
3     2154
2     1504
Name: Score, dtype: int64

In [33]:
cleaned_df = cleaned_df.dropna(subset=['Text'])  # Drop rows with missing values in 'Text' column

X = cleaned_df['Text']  # Input features (Text column)
y = cleaned_df['Score']  # Target variable

# Convert text data to sequences
vectorizer   = CountVectorizer(max_features=5000)  # Set maximum 5000 features
X_vectorized = vectorizer.fit_transform(X)
X_sequences  = pad_sequences(X_vectorized.toarray())

In [34]:
# Perform train-test split
x_train,x_test, y_train, y_test = train_test_split(X_sequences, y, test_size=0.2, random_state=42)

In [35]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(22737, 5000) (5685, 5000) (22737,) (5685,)


build LSTM

In [36]:
# Create a Logistic Regression model
LOGREG_Model = LogisticRegression()

In [37]:
# Train the model
LOGREG_Model.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [38]:
# Predict on the test set
y_pred = LOGREG_Model.predict(x_test)

In [39]:
# Calculate the classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           1       0.59      0.51      0.55       518
           2       0.27      0.23      0.25       286
           3       0.28      0.26      0.27       419
           4       0.37      0.26      0.30       831
           5       0.80      0.88      0.84      3631

    accuracy                           0.68      5685
   macro avg       0.46      0.43      0.44      5685
weighted avg       0.65      0.68      0.66      5685



In [40]:
# Get the classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Convert the report to a DataFrame
report_df = pd.DataFrame(report).transpose()

# Set the desired plot colors
colors = ['#8A2BE2', '#8A2BE2', '#8A2BE2', '#8A2BE2']

# Create the precision plot
precision_fig = go.Figure(data=go.Bar(x=report_df.index, y=report_df['precision'], marker=dict(color=colors)))
precision_fig.update_layout(title='Precision', plot_bgcolor='black', paper_bgcolor='black', font_color='white')
precision_fig.show()

# Create the recall plot
recall_fig = go.Figure(data=go.Bar(x=report_df.index, y=report_df['recall'], marker=dict(color=colors)))
recall_fig.update_layout(title='Recall', plot_bgcolor='black', paper_bgcolor='black', font_color='white')
recall_fig.show()

# Create the f1-score plot
f1_score_fig = go.Figure(data=go.Bar(x=report_df.index, y=report_df['f1-score'], marker=dict(color=colors)))
f1_score_fig.update_layout(title='F1-Score', plot_bgcolor='black', paper_bgcolor='black', font_color='white')
f1_score_fig.show()

# Create the support plot
support_fig = go.Figure(data=go.Bar(x=report_df.index, y=report_df['support'], marker=dict(color=colors)))
support_fig.update_layout(title='Support', plot_bgcolor='black', paper_bgcolor='black', font_color='white')
support_fig.show()

In [43]:
from tensorflow.keras.utils import to_categorical
# Determine the number of unique classes in y_train
num_classes = len(y_train.unique())
# Encode the target variable
y_train_encoded = to_categorical(y_train, num_classes=6)
y_test_encoded= to_categorical(y_test,num_classes=6)
# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=100, input_length=5000))
model.add(LSTM(units=100))
model.add(Dense(units=6, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train_encoded, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test_encoded)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

Epoch 1/10
711/711 [==============================] - 111s 153ms/step - loss: 1.1496 - accuracy: 0.6369
Epoch 2/10
711/711 [==============================] - 108s 152ms/step - loss: 1.1387 - accuracy: 0.6379
Epoch 3/10
711/711 [==============================] - 105s 148ms/step - loss: 1.1377 - accuracy: 0.6379
Epoch 4/10
711/711 [==============================] - 105s 148ms/step - loss: 1.1369 - accuracy: 0.6379
Epoch 5/10
711/711 [==============================] - 105s 148ms/step - loss: 1.1370 - accuracy: 0.6379
Epoch 6/10
711/711 [==============================] - 105s 148ms/step - loss: 1.1366 - accuracy: 0.6379
Epoch 7/10
711/711 [==============================] - 105s 148ms/step - loss: 1.1362 - accuracy: 0.6379
Epoch 8/10
711/711 [==============================] - 105s 148ms/step - loss: 1.1366 - accuracy: 0.6379
Epoch 9/10
711/711 [==============================] - 106s 149ms/step - loss: 1.1360 - accuracy: 0.6379
Epoch 10/10
178/178 [==============================] - 11s 61ms/